# E14 - Grupo 5

# 0. Cargue de datos y librerías

<a href="https://colab.research.google.com/github/dljaimes/AdvancedMethodsinDataAnalysis1Grupo_5/blob/main/E_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from livelossplot import PlotLossesKeras
import keras
from keras import backend as K
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from livelossplot import PlotLossesKeras

In [11]:
# Cargue del dataset:

dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)

In [13]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


## Exercise 14.1

* Removemos caracteres especiales

In [14]:
# Se quitan caractéres especiales sobre cada texto:

def pre_process(text):
    # lowercase
    text = text.lower()
    # tags
    text = re.sub('&lt;/?.*?&gt;',' &lt;&gt; ',text)
    # special characters and digits
    text=re.sub('(\\d|\\W)+',' ',text)
    # remove punctuation
    #text = re.sub('[.;:!\'?,\"()\[\]]', '', text)
    #text = [REPLACE.sub('', line) for line in text]
    return text

# Se crea una columna con 
dataTraining['plot_low']=dataTraining['plot'].apply(lambda x:pre_process(x))


In [15]:
dataTraining.columns

Index(['year', 'title', 'plot', 'genres', 'rating', 'plot_low'], dtype='object')

* Removemos Stopwords

In [21]:
# Se descargan los Stopwords:
import nltk  as nltk  
nltk.download('stopwords')
nltk.corpus.stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Geotrend\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Geotrend\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
# Se quitan los stopwords:
english_stop_words=stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

dataTraining['plot_low_rm'] = remove_stop_words(dataTraining['plot_low'])

from nltk.stem.porter import PorterStemmer
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

dataTraining['plot_low_rm_stem'] = get_stemmed_text(dataTraining['plot_low_rm'])

In [19]:
#Revisamos dataframe consolidado 
dataTraining[['plot','plot_low','plot_low_rm_stem']]

,plot,plot_low,plot_low_rm_stem
3107,most is the story of a single father who takes...,most is the story of a single father who takes...,stori singl father take eight year old son wor...
900,a serial killer decides to teach the secrets o...,a serial killer decides to teach the secrets o...,serial killer decid teach secret satisfi caree...
6724,"in sweden , a female blackmailer with a disfi...",in sweden a female blackmailer with a disfigur...,sweden femal blackmail disfigur facial scar me...
4704,"in a friday afternoon in new york , the presi...",in a friday afternoon in new york the presiden...,friday afternoon new york presid tredway corpo...
2582,"in los angeles , the editor of a publishing h...",in los angeles the editor of a publishing hous...,lo angel editor publish hous carol hunnicut go...
...,...,...,...
8417,""" our marriage , their wedding . "" it ' s l...",our marriage their wedding it s lesson number...,marriag wed lesson number one newli engag coup...
1592,"the wandering barbarian , conan , alongside ...",the wandering barbarian conan alongside his go...,wander barbarian conan alongsid goofi rogu pal...
1723,"like a tale spun by scheherazade , kismet fol...",like a tale spun by scheherazade kismet follow...,like tale spun scheherazad kismet follow remar...
7605,"mrs . brisby , a widowed mouse , lives in a...",mrs brisby a widowed mouse lives in a cinder b...,mr brisbi widow mous live cinder block childre...


In [23]:
# Se separan las variables de salida (y) y el texto transformado (sin stopwords y stemmizado):

plots = dataTraining['plot_low_rm_stem']
y = (dataTraining['rating'] >= dataTraining['rating'].mean()).astype(int) # se discretiza la variable 1 si la calificacion es >= promedio 0 dlc.

In [24]:
y

3107    1
900     0
6724    1
4704    1
2582    1
       ..
8417    0
1592    0
1723    0
7605    1
215     1
Name: rating, Length: 7895, dtype: int32

* Toquenización

In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
# Tokenización por palabra:
count_ = Tokenizer()
count_.fit_on_texts(plots)

In [27]:
vocabulary = count_.word_index

In [28]:
'major' in vocabulary.keys()

True

* Transformación de cada texto en palabras

In [29]:
# Se transforma cada texto en palabras de acuerdo con su diccionario de índices:
frases_num = count_.sequences_to_texts(plots)
matriz_pad =  pad_sequences(frases_num)

In [30]:
matriz_pad

array([], shape=(7895, 0), dtype=int32)

In [31]:
# Se crea diccionario con el índice de cada letra/palabra:
voc = set(''.join(plots))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}

In [32]:
vocabulary

{'ø': 1,
 'ï': 2,
 'ñ': 3,
 'ü': 4,
 ' ': 5,
 'q': 6,
 's': 7,
 'r': 8,
 'k': 9,
 'á': 10,
 '¹': 11,
 'à': 12,
 't': 13,
 'ê': 14,
 'e': 15,
 'ç': 16,
 'â': 17,
 'j': 18,
 'å': 19,
 'z': 20,
 'ò': 21,
 '½': 22,
 'p': 23,
 'h': 24,
 'i': 25,
 'w': 26,
 'g': 27,
 'f': 28,
 'm': 29,
 'v': 30,
 'o': 31,
 'd': 32,
 'x': 33,
 'ú': 34,
 'é': 35,
 'ë': 36,
 'y': 37,
 'ʼ': 38,
 'è': 39,
 'û': 40,
 'ù': 41,
 'a': 42,
 'n': 43,
 'í': 44,
 'ä': 45,
 'l': 46,
 'b': 47,
 'c': 48,
 'ö': 49,
 'u': 50,
 'ô': 51,
 'ó': 52}

In [33]:
plots[0].split()[:10]

['major',
 'benson',
 'winifr',
 'payn',
 'discharg',
 'marin',
 'payn',
 'killin',
 'machin',
 'war']

In [34]:
max_url_len = 540
X = [x[:max_url_len] for x in plots]
# Convert characters to int and pad
X = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in plots.str.split()]

In [35]:
X[0][:10]

[]

In [36]:
dataTraining[['plot','plot_low','plot_low_rm_stem']].head(10)

,plot,plot_low,plot_low_rm_stem
3107,most is the story of a single father who takes...,most is the story of a single father who takes...,stori singl father take eight year old son wor...
900,a serial killer decides to teach the secrets o...,a serial killer decides to teach the secrets o...,serial killer decid teach secret satisfi caree...
6724,"in sweden , a female blackmailer with a disfi...",in sweden a female blackmailer with a disfigur...,sweden femal blackmail disfigur facial scar me...
4704,"in a friday afternoon in new york , the presi...",in a friday afternoon in new york the presiden...,friday afternoon new york presid tredway corpo...
2582,"in los angeles , the editor of a publishing h...",in los angeles the editor of a publishing hous...,lo angel editor publish hous carol hunnicut go...
9784,two drifters are passing through a western tow...,two drifters are passing through a western tow...,two drifter pass western town news come local ...
8804,two gay men on a date are murdered the night b...,two gay men on a date are murdered the night b...,two gay men date murder night halloween west h...
3501,adult huw morgan is about to move away from th...,adult huw morgan is about to move away from th...,adult huw morgan move away south wale coal min...
7145,bertram pincus is a man whose people skills le...,bertram pincus is a man whose people skills le...,bertram pincu man whose peopl skill leav much ...
4067,roy munson was raised to be the best bowler in...,roy munson was raised to be the best bowler in...,roy munson rais best bowler world train earli ...


In [38]:
dataTraining['plot_low_rm_stem'].str.len().describe()

count    7895.000000
mean      424.891577
std       275.441225
min         0.000000
25%       236.000000
50%       371.000000
75%       540.000000
max      4930.000000
Name: plot_low_rm_stem, dtype: float64

In [39]:
len(dataTraining['plot'].tolist())

7895

In [40]:
X_pad = sequence.pad_sequences(X, maxlen=max_url_len)
X_pad

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 24, 43, 24],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [41]:
X_pad.shape

(7895, 540)

## Exercise 14.2.

* Creación de una red neuronal LSTM 

In [122]:
from keras.layers import Bidirectional
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_url_len))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Se especifica la función de pérdida, optimizador y la métrica de evaluación
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 540, 128)          6784      
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 286,465
Trainable params: 286,465
Non-trainable params: 0
_________________________________________________________________


In [123]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [124]:
model.fit(X_train, y_train, validation_data=[X_test, y_test], 
          batch_size=64, epochs=2, verbose=1)
          #callbacks=[PlotLossesKeras()])

Epoch 1/2
99/99 [==============================] - 1539s 15s/step - loss: 0.6909 - accuracy: 0.5202 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/2
99/99 [==============================] - 1649s 17s/step - loss: 0.6880 - accuracy: 0.5311 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [125]:
from sklearn.metrics import accuracy_score

In [126]:
pred_ = model.predict_classes(X_pad)

In [127]:
import numpy as np
y_real = np.array(y).reshape(-1,1)

* Accuracy sobre toda la base

In [128]:
accuracy_score(pred_, y_real)

0.5400886637112097

* Accuracy sobre test

In [129]:
# Se calcula el accuracy sobre la base de Test:
pred_test = model.predict_classes(X_test)
accuracy_score(pred_test, y_test)

0.5174160861304623

El Accuracy obtenido en la base de test creada equivale a **0.52**